<a href="https://colab.research.google.com/github/Debora-Simoes/IA901-2023S1/blob/main/projetos/Reconhecimento_acao_humana_imagem_drone/notebooks/Creating_labels_from_folders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Going to our repository clone and installing dependencies

from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/IA901_Projeto/
# !pip install -qr requirements.txt

In [ ]:
!ls

Dados_filtrados		Entrega1.gdoc	      Pre_processing.ipynb  yolov7
Dados.ipynb		Entrega_2.gdoc	      Projeto_IA.ipynb	    Yolov7_ref
Dados_pre_processing	Imagens_Dataset.gdoc  references.gsheet
_Dataset_NTUT_4K_drone	Notebooks	      tmp_data


In [ ]:
# Coloque o caminho pros dados (pasta onde tem o treino/val/test)
DATA_DIR = "Dados_filtrados"

In [ ]:
import os

## Checando a pasta


In [ ]:
os.listdir(DATA_DIR)

['Teste', 'Treino', 'Val']

In [ ]:
for split in os.listdir(DATA_DIR):
  print(os.listdir(DATA_DIR + '/' + split))

['labels', 'images']
['labels', 'images']
['labels', 'images']


Precisamos que as pastas sejam "images" e "labels". Então vamos alterar o nome das pastas.

In [ ]:
# for split in os.listdir(DATA_DIR):
#   os.rename(DATA_DIR+'/'+split+'/'+'Imagens/', DATA_DIR+'/'+split+'/'+'images/')

for split in os.listdir(DATA_DIR):
  print(os.listdir(DATA_DIR + '/' + split))

['labels', 'images']
['labels', 'images']
['labels', 'images']


## Agora processando os labels

Precisamos que na pasta "labels" tenhamos um arquivo txt para cada imagem, com as informações de, ids das classes presentes naquela imagem e os bouding box normalizados.

In [ ]:
!ls Dados_filtrados/Treino/labels

label_train.txt


In [ ]:
import pandas as pd

# Primeiro vamos mapear todos os arquivos de label para extrair todas as labels possíveis
dfs = {} # Dicionário em que cada chave é um dataset
for split in os.listdir(DATA_DIR):
  for file in os.listdir(DATA_DIR+'/'+split+'/'+'labels/'):
    dfs[split] = pd.read_csv(DATA_DIR + '/' + split + '/' + 'labels' + '/' + file)

print(dfs.keys()) # Tenhos datasets de teste, treino e val em cada uma de suas respectivas keys

dict_keys(['Teste', 'Treino', 'Val'])


In [ ]:
for key in dfs.keys():
  print(f"Visualizando labels únicas do dataset de {key}")
  print(dfs[key]['label'].unique())

Visualizando labels únicas do dataset de Teste
['walk' 'stand' 'sit' 'riding']
Visualizando labels únicas do dataset de Treino
['walk' 'riding' 'stand' 'sit']
Visualizando labels únicas do dataset de Val
['walk' 'riding' 'stand' 'sit']


In [ ]:
# Dessa forma podemos fazer um mapeamento de labels pegando cada valor único e levando pra um ID único

map_label = {}

for i, label in enumerate(dfs[key].label.unique()):
  map_label[label] = i

map_label # Esse será o nosso mapeamento

{'walk': 0, 'riding': 1, 'stand': 2, 'sit': 3}

In [ ]:
# Vamos agr gerar os bounding box normalizados
for key in dfs.keys():
  df = dfs[key]
  df['x_center'] = (df['xmin'] + df['xmax'])/2
  df['y_center'] = (df['ymin'] + df['ymax'])/2
  df['w'] = df['xmax'] - df['xmin']
  df['h'] = df['ymax'] - df['ymin']

  img_h, img_w, num_channels = (2160, 3840, 3)

  df['x_center'] = df['x_center']/img_w
  df['w'] = df['w']/img_w
  df['y_center'] = df['y_center']/img_h
  df['h'] = df['h']/img_h

  df['maped_labels'] = df.label.map(map_label)

In [ ]:
dfs[key].head()

,image,xmin,ymin,xmax,ymax,label,x_center,y_center,w,h,maped_labels
0,Drone_023.mp4_t-16.166667.jpg,95.272226,884.365809,219.655410,1014.071691,walk,0.041006,0.439453,0.032391,0.060049,0
1,Drone_005.mp4_t-107.666667.jpg,871.345646,2039.402110,967.598945,2117.901524,walk,0.239446,0.962339,0.025066,0.036342,0
2,Drone_023.mp4_t-27.333333.jpg,1489.951757,1757.895221,1590.516885,1861.130515,walk,0.401103,0.837737,0.026189,0.047794,0
3,Drone_005.mp4_t-153.333333.jpg,1170.237467,687.186401,1246.226913,753.024619,walk,0.314644,0.333382,0.019789,0.030481,0
4,Drone_005.mp4_t-54.666667.jpg,2687.733888,1447.093596,2754.261954,1564.137931,walk,0.708593,0.697044,0.017325,0.054187,0


In [ ]:
dfs[key].image.values

array(['Drone_023.mp4_t-16.166667.jpg', 'Drone_005.mp4_t-107.666667.jpg',
       'Drone_023.mp4_t-27.333333.jpg', ..., 'Drone_023.mp4_t-4.jpg',
       'Drone_031.mp4_t-31.166667.jpg', 'Drone_023.mp4_t-29.166667.jpg'],
      dtype=object)

In [ ]:
key

'Val'

In [ ]:
lens = {}
for split in os.listdir(DATA_DIR):
  lens[split] = len(os.listdir(DATA_DIR + '/' + split + '/images/'))
  print(f'Na pasta de imagens de {split} temos {lens[split]} imagens')

Na pasta de imagens de Teste temos 1266 imagens
Na pasta de imagens de Treino temos 1676 imagens
Na pasta de imagens de Val temos 1006 imagens


In [ ]:
for key in dfs.keys():
  print(f"Na pasta de labels de {key} temos {dfs[key].shape[0]} labels")

Na pasta de labels de Teste temos 6271 labels
Na pasta de labels de Treino temos 8479 labels
Na pasta de labels de Val temos 2123 labels


In [ ]:
for split in os.listdir(DATA_DIR):
  set_ = set(os.listdir(DATA_DIR + '/' + split + '/images/'))

  labels_list = []
  for img in dfs[split].image.values:
    if('.jpg' not in img):
      img = img + '.jpg'
    labels_list.append(img)
  
  set_labels_ = set(labels_list)

  print(f"Para o split {split} as imagens das pastas estão contidas na lista de labels? {set_.issubset(set_labels_)}")

Para o split Teste as imagens das pastas estão contidas na lista de labels? True
Para o split Treino as imagens das pastas estão contidas na lista de labels? False
Para o split Val as imagens das pastas estão contidas na lista de labels? True


In [ ]:
count = 0
# Ok, como alguns arquivo tem a extensão e outros não, primeiro vamos checar se todos os arquivos existem
for img in dfs['Treino'].image.values:
  if('.jpg' not in img):
    img = img + '.jpg'
  pathing = DATA_DIR + '/' + key + '/' + 'images/' + img
  if(not os.path.isfile(pathing)):
    print(f'{img} does not exist!')
    count += 1
count

Drone_005.mp4_t-118.333333.jpg does not exist!
Drone_023.mp4_t-8.5.jpg does not exist!
Drone_005.mp4_t-41.jpg does not exist!
Drone_005.mp4_t-59.166667.jpg does not exist!
Drone_023.mp4_t-22.666667.jpg does not exist!
Drone_031.mp4_t-287.jpg does not exist!
Drone_031.mp4_t-117.166667.jpg does not exist!
Drone_031.mp4_t-278.666667.jpg does not exist!
Drone_005.mp4_t-16.jpg does not exist!
Drone_031.mp4_t-72.833333.jpg does not exist!
Drone_005.mp4_t-145.666667.jpg does not exist!
Drone_005.mp4_t-64.jpg does not exist!
Drone_023.mp4_t-30.833333.jpg does not exist!
Drone_005.mp4_t-61.833333.jpg does not exist!
Drone_005.mp4_t-56.5.jpg does not exist!
Drone_005.mp4_t-147.333333.jpg does not exist!
Drone_031.mp4_t-302.833333.jpg does not exist!
Drone_023.mp4_t-26.5.jpg does not exist!
Drone_023.mp4_t-33.333333.jpg does not exist!
Drone_023.mp4_t-77.666667.jpg does not exist!
Drone_005.mp4_t-114.jpg does not exist!
Drone_005.mp4_t-125.666667.jpg does not exist!
Drone_023.mp4_t-74.5.jpg does 

1884

In [ ]:
columns = ['maped_labels', 'x_center', 'y_center','w','h']


for fname in df_filt.image.unique():
  df_filt2 = df_filt[df_filt.image == fname]
  fname = fname[:-4]
  df_filt2[columns].to_csv('./tmp_data/train/labels/' + fname + '.txt', index = False, header = False, sep = ' ')
  df_filt2[columns].to_csv('./tmp_data/val/labels/' + fname + '.txt', index = False, header = False, sep = ' ')